In [1]:
import math
import numpy as np
import pandas as pd
from scipy import optimize
import matplotlib.pyplot as plt
import seaborn as sns
from numpy.random import default_rng
from IPython.display import Image
import simpy

# Simulation Modeling

## Using Discrete Event Simulation to help plan Purchase Requisition for Project Coordinators

Send application --> application Review --> Create PR on Requisition Management System --> if Amazon, Place order on amazon with Req. # --> Level 1 approval --> Level 2 approval --> issue PO to merchants --> items delivered --> PR closed

Model components:
* The entities are applications
* Entities are created by email
* Resources at different stages - Employees, Project Coordinators, Level aprrovers - Directors, level 2 approvers - Finance, Indirect Buyer, Merchants, Delivery service
* Entities flow through the different stages of the process, and at each stage; they...... (rewrite)

Our model should handle:
* Uncertainty in the processing times of the individual steps above
* Uncertainty in the application arrival times
* Finite number of different types of resources
* Estimating key process metrics such as wait times, total time in for completion, resource utilisation

The advantages of model .......


**Tools** - DES Software - Simpy


## Model 1: Generate PR applications

Model 1 generates PR Applications. It generates a new Application every n minutes

In [2]:
def PR_application(env, interapplication_time=5.0):
    """Generate Applications according to a fixed time arrival process"""
    
    # Counter to keep track of number of Applications generated and to serve as a unique Application ID
    application = 0
    
    # Infinite loop for generating Applications
    while True:
        
        # Generate next interapplication time
        iat = interapplication_time
        
        # yield 'timeout' event
        yield env.timeout(iat)
        
        # generate new application
        application += 1
        
        print(f"Application {application} created at time {env.now}")
        

In [3]:
# Initialize a simulation environment
env1 = simpy.Environment()

# Create and start process generator, and add it to the env (for 1 workday - [60x8] mins )
runtime = 60 * 8
interapplication_time = 5.0
env1.process(PR_application(env1, interapplication_time))

# Run the simulation
env1.run(until=runtime)

Application 1 created at time 5.0
Application 2 created at time 10.0
Application 3 created at time 15.0
Application 4 created at time 20.0
Application 5 created at time 25.0
Application 6 created at time 30.0
Application 7 created at time 35.0
Application 8 created at time 40.0
Application 9 created at time 45.0
Application 10 created at time 50.0
Application 11 created at time 55.0
Application 12 created at time 60.0
Application 13 created at time 65.0
Application 14 created at time 70.0
Application 15 created at time 75.0
Application 16 created at time 80.0
Application 17 created at time 85.0
Application 18 created at time 90.0
Application 19 created at time 95.0
Application 20 created at time 100.0
Application 21 created at time 105.0
Application 22 created at time 110.0
Application 23 created at time 115.0
Application 24 created at time 120.0
Application 25 created at time 125.0
Application 26 created at time 130.0
Application 27 created at time 135.0
Application 28 created at time

### Poisson Arrival Process

In [4]:
from numpy.random import default_rng
rg = default_rng(seed=4470)

In [5]:
from scipy.stats import expon
mean_interapplication_time = interapplication_time 

In [6]:
def Purchase_application_random_1(env, mean_interapplication_time=5.0, rg=default_rng(0)):
    """Generate applications according to a Poisson arrival process"""
    
    # Counter to keep track of number of applications generated and to serve as a unique application ID
    application = 0
    
    # Infinite loop for generating applications
    while True:
        
        # Generate next interapplication time
        iat = rg.exponential(mean_interapplication_time)
        
        # yield 'timeout' event
        yield env.timeout(iat)
        
        # generate new application
        application += 1
        
        print(f"application {application} created at time {env.now}")
        

In [7]:
# Initialize a simulation environment
env2 = simpy.Environment()

# Create and start process generator, and add it to the env (for 1 workday - [60x8] mins )
runtime = 60 * 8
interapplication_time = 5.0
env2.process(Purchase_application_random_1(env2, interapplication_time))

# Run the simulation
env2.run(until=runtime)

application 1 created at time 3.3996595198445476
application 2 created at time 8.49764502717387
application 3 created at time 8.596678340119148
application 4 created at time 8.60802497352529
application 5 created at time 11.35973933672053
application 6 created at time 19.509441510012458
application 7 created at time 22.877356273348617
application 8 created at time 26.653863062475573
application 9 created at time 40.7377929578542
application 10 created at time 71.02655836006706
application 11 created at time 87.45869964953577
application 12 created at time 87.46513840120988
application 13 created at time 98.81061172254442
application 14 created at time 99.17310014714388
application 15 created at time 104.52010165798048
application 16 created at time 108.76476680570316
application 17 created at time 124.51431006939562
application 18 created at time 126.28436028036187
application 19 created at time 127.8199159785036
application 20 created at time 135.28101304413372
application 21 created 

## Model 2: Simplified version with delay process and one resource

* Add a Coordinator resource
* Create a new process function with new delay and resourse
* Modify application process to create PR creation for each PR application

In [8]:
rg.integers(1,high=301)

280

In [9]:
def Purchase_application_Simplified(env, employee, mean_precreation_time, mean_creation_time, mean_postcreation_time, coordinator):
    """Process function modeling how a PR flows through system."""
    print(f"{employee} applied for PR at {env.now:.4f}")
    
    # Yield for the precreation time
    yield env.timeout(rg.exponential(mean_precreation_time))
    
    # Request project coordinator to create PR
    with coordinator.request() as request:
        print(f"{employee} requested coordinator at {env.now:.4f}")
        yield request
        print(f"{employee} received PR at {env.now:.4f}")
        yield env.timeout(rg.normal(mean_creation_time, 0.5))
        
    # Yield for the postcreation time
    yield env.timeout(mean_postcreation_time)
    
    # Process over
    print(f"{employee} received P.O. at {env.now:.4f}")

**Combining process and arrival**

In [10]:
def Purchase_application_random_2(env, mean_interapplication_time, mean_precreation_time, mean_creation_time,
                              mean_postcreation_time, coordinator,  rg=default_rng(0)):
    """Generate applications according to a Poisson arrival process"""
    
    # Counter to keep track of number of applications generated and to serve as a unique application ID
    application = 0
    
    # Infinite loop for generating applications
    while True:
        
        # Generate next interarrival time
        iat = rg.exponential(mean_interapplication_time)
        
        # yield 'timeout' event
        yield env.timeout(iat)
        
        # generate new application
        application += 1
        
        print(f"application {application} created at time {env.now}")
        
        # Register the process with the simulation environment
        env.process(Purchase_application_Simplified(env, 'Employee{}'.format(application), mean_precreation_time,
                                                    mean_creation_time, mean_postcreation_time, coordinator))
        
        
        

In [11]:
# Initialize a simulation environment
env3 = simpy.Environment()

# Set input values
mean_interapplication_time = 3.0
mean_precreation_time = 5.0
mean_creation_time = 30
mean_postcreation_time = 120
num_coordinators = 4

# Create Coordinator resource
coordinator = simpy.Resource(env3, num_coordinators)

# register new application process
env3.process(Purchase_application_random_2(env3, mean_interapplication_time, mean_precreation_time, mean_creation_time,
                                           mean_postcreation_time, coordinator))

# Run the simulation
runtime = 50
env3.run(until=runtime)

application 1 created at time 2.039795711906729
Employee1 applied for PR at 2.0398
Employee1 requested coordinator at 2.4779
Employee1 received PR at 2.4779
application 2 created at time 5.098587016304323
Employee2 applied for PR at 5.0986
application 3 created at time 5.158007004071489
Employee3 applied for PR at 5.1580
application 4 created at time 5.164814984115174
Employee4 applied for PR at 5.1648
application 5 created at time 6.815843602032318
Employee5 applied for PR at 6.8158
Employee2 requested coordinator at 6.9273
Employee2 received PR at 6.9273
Employee4 requested coordinator at 9.7762
Employee4 received PR at 9.7762
Employee3 requested coordinator at 10.0449
Employee3 received PR at 10.0449
application 6 created at time 11.705664906007474
Employee6 applied for PR at 11.7057
Employee6 requested coordinator at 11.9072
application 7 created at time 13.72641376400917
Employee7 applied for PR at 13.7264
Employee5 requested coordinator at 14.1935
application 8 created at time 15

## Model 3: The PR application model - Version 0.01

* the SimPy environment
* resource capacity related inputs
* data structures to store data collected as PRs flow through the system
* the SimPy resources for modeling the various types of staff modeled
* process methods corresponding to processing times in each stage in the PR flow diagram

In [12]:
class purchase_requisition(object):
    def __init__(self, env, num_coordinators, num_approvers, num_indirectbuyers, rg):
        # Simulation environment
        self.env = env
        self.rg = rg
        
        # list to hold timestamps dictionaries (one per PR)
        self.timestamps_list = []
        # list to hold PR queues (time, queue)
        self.pr_queue = [(0.0, 0.0)]
        self.postapproval_queue = [(0.0, 0.0)]
        
        # Create resources
        self.coordinator = simpy.Resource(env, num_coordinators)
        self.approver = simpy.Resource(env, num_approvers)
        self.indirectbuyer = simpy.Resource(env, num_indirectbuyers)
        
    # Process methods
    def create_PR(self, application):
        yield self.env.timeout(self.rg.exponential(1.0))
        
    def approve(self, application):
        yield self.env.timeout(self.rg.normal(4.0, 0.5))
        
    def create_order(self, application):
        yield self.env.timeout(self.rg.exponential(1.0))
        
    def create_po(self, application):
        yield self.env.timeout(self.rg.normal(2.0, 0.5))
        

**The create_pr function**

* the simulation environment
* the PR I.D
* the PR object (created from the PRapplication class)
* the random number generator

In [13]:
def createPR(env, application, purchase_requisition, pct_amazon, rg):
    # An application is received
    app_request_ts = env.now
    
    # release coordinator to create PR
    with purchase_requisition.coordinator.request() as request:
        yield request
        # With coordinator, create application
        got_coordinator_ts = env.now
        
        # increase and updated PR queue
        purchase_requisition.pr_queue.append((env.now, purchase_requisition.pr_queue[-1][1] + 1))
        
        yield env.process(purchase_requisition.create_PR(application))
        release_coordinator_ts = env.now
        
    # Request approver to approve PR
    with purchase_requisition.approver.request() as request:
        yield request
        got_approver_ts = env.now
        yield env.process(purchase_requisition.approve(application))
        release_approver_ts = env.now
        
        # decrease and update PR queue
        purchase_requisition.pr_queue.append((env.now, purchase_requisition.pr_queue[-1][1] - 1))
        
        # increase and update post approval queue
        purchase_requisition.postapproval_queue.append((env.now, purchase_requisition.postapproval_queue[-1][1] + 1))
        
    # if amazon order, request coordinator to place order on amazon
    if rg.random() < pct_amazon:
        with purchase_requisition.coordinator.request() as request:
            yield request
            got_coordinator2_ts = env.now
            yield env.process(purchase_requisition.create_order(application))
            release_coordinator2_ts = env.now      
    else:
        got_coordinator2_ts = pd.NA
        release_coordinator2_ts = pd.NA
    
    # Request indirect buyer to issue PO
    with purchase_requisition.indirectbuyer.request() as request:
        yield request
        got_indirectbuyer_ts = env.now
        yield env.process(purchase_requisition.create_po(application))
        release_indirectbuyer_ts = env.now
        
        # decrease and update post approval queue
        purchase_requisition.postapproval_queue.append((env.now, purchase_requisition.postapproval_queue[-1][1] -1))
        
        
    exit_system_ts = env.now
    print(f'PO has been issued for purchase requisition application {application}')
    
    # Create dictionary of timestamps
    timestamps = {'application_id': application,
                  'app_request_ts': app_request_ts,
                  'got_coordinator_ts': got_coordinator_ts,
                  'release_coordinator_ts': release_coordinator_ts,
                  'got_approver_ts': got_approver_ts,
                  'release_approver_ts': release_approver_ts,
                  'got_coordinator2_ts': got_coordinator2_ts,
                  'release_coordinator2_ts': release_coordinator2_ts,
                  'got_indirectbuyer_ts': got_indirectbuyer_ts,
                  'release_indirectbuyer_ts': release_indirectbuyer_ts,
                  'exit_system_ts': exit_system_ts}
    
    purchase_requisition.timestamps_list.append(timestamps)
                  

**The run_pr function**

Function input
* the simulation environment
* the purchase requisition object
* the mean interapplication time
* the percentage of requisitions containing items from amazon
* the random number generator
* stopping condition for the simulation

In [14]:
def run_pr(env, purchase_requisition, mean_interapplication_time, pct_amazon, rg,
           stoptime=simpy.core.Infinity, max_applications=simpy.core.Infinity):
    
    # application counter/ID
    application = 0
    
    # Loop for generating applications
    while env.now < stoptime and application < max_applications:
        
        iat = rg.exponential(mean_interapplication_time)
        yield env.timeout(iat)
        application += 1
        
        print(f"Purchase Requisition application {application} created at time {env.now}")
        
        env.process(createPR(env, application, purchase_requisition, pct_amazon, rg))
        

In [15]:
def main():
    # input parameters
    applications_per_hour = 20
    mean_interapplication_time = 1.0 / (applications_per_hour / 60.0)
    pct_amazon = 0.4
    
    # Random number generator seed
    rg = default_rng(seed=4470)
    
    # Resource capacity levels
    num_coordinators = 5
    num_approvers = 10
    num_indirectbuyers = 2
    
    # Hours of operation
    stoptime = 8*60
    
    # Create simulation environment
    env = simpy.Environment()
    
    # create purchase requisition to simulate
    requisition = purchase_requisition(env, num_coordinators, num_approvers, num_indirectbuyers, rg)
    
    # Register PR creation function
    env.process(run_pr(env, requisition, mean_interapplication_time, pct_amazon, rg, stoptime=stoptime))
    # Run simulation
    env.run()
    
    # Output log files
    pr_application_log_df = pd.DataFrame(requisition.timestamps_list)
    pr_application_log_df.to_csv('./output/pr_application_log_df.csv', index=False)
    
    pr_queue_df = pd.DataFrame(requisition.pr_queue, columns=['ts', 'queue'])
    pr_queue_df.to_csv('./output/pr_queue_df.csv', index=False)
    
    postapproval_queue_df = pd.DataFrame(requisition.postapproval_queue, columns=['ts', 'queue'])
    postapproval_queue_df.to_csv('./output/postapproval_queue_df.csv', index=False)
    
    # Simulation end time
    end_time = env.now
    print(f"Simulation ended at time {end_time}")
    return (end_time)
    

In [16]:
pr_end_time = main()

Purchase Requisition application 1 created at time 2.332475473171058
Purchase Requisition application 2 created at time 2.5953536130475916
Purchase Requisition application 3 created at time 3.6925802144818
Purchase Requisition application 4 created at time 5.807172399828623
Purchase Requisition application 5 created at time 5.928094344228452
Purchase Requisition application 6 created at time 6.20560255122298
Purchase Requisition application 7 created at time 6.415200739403505
PO has been issued for purchase requisition application 1
PO has been issued for purchase requisition application 3
Purchase Requisition application 8 created at time 11.24151890448196
PO has been issued for purchase requisition application 2
PO has been issued for purchase requisition application 7
PO has been issued for purchase requisition application 6
PO has been issued for purchase requisition application 5
PO has been issued for purchase requisition application 4
Purchase Requisition application 9 created a